### Extract Lyrics Embeddings

In [4]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import scipy as sp

In [7]:
# import dataset
data = pd.read_csv('./spotify_data.csv', index_col=[0])
lyrics_data = pd.DataFrame(data['lyrics'])
lyrics_data['embedding'] = ''
lyrics_data['lyrics'] = lyrics_data['lyrics'].apply(lambda e: e.replace('"',''))

In [8]:
# define model
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig, pipeline
import torch

config = RobertaConfig.from_pretrained("SamLowe/roberta-base-go_emotions")
config.output_hidden_states = True
tok = RobertaTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
model = RobertaModel.from_pretrained("SamLowe/roberta-base-go_emotions", config=config)

# extract embedding
lyrics_data['embedding'] = lyrics_data['lyrics'].apply(lambda e: np.array(model(torch.tensor([tok.encode(e)[:512]])).pooler_output.squeeze().detach()))

# lyrics_data['embedding'] = lyrics_data['lyrics'].apply(lambda e: model(torch.tensor([tok.encode(e)])).pooler_output.squeeze())
lyrics_data

Some weights of RobertaModel were not initialized from the model checkpoint at SamLowe/roberta-base-go_emotions and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (674 > 512). Running this sequence through the model will result in indexing errors


,lyrics,embedding
160,Aah Aah The games you played were never fun Yo...,"[-0.03261699, 0.090100184, -0.76937836, -0.350..."
445,The lights go out and I can't be saved Tides t...,"[-0.11807676, 0.19294278, -0.6089342, -0.36304..."
763,"'Cause you're a sky, 'cause you're a sky full ...","[-0.4585999, -0.23294687, -0.59005123, -0.6478..."
856,I will not make the same mistakes that you did...,"[-0.59562796, -0.06940222, -0.7404203, -0.5954..."
1169,Heyy Heeey Heey Your lipstick stains On the fr...,"[0.5713806, 0.72226596, -0.57876605, 0.1377586..."
...,...,...
16203,"Yeah, you! Yeah, you! I used to wanna be Livin...","[-0.115405865, 0.28116122, -0.5480692, 0.32812..."
16284,"If you don't wanna see me Did a full 180, craz...","[-0.5470382, 0.48770085, -0.71977407, -0.50465..."
16771,"Nah, nah, nah Cake by the ocean Oh, no See you...","[-0.1422666, 0.17752872, -0.6433116, 0.1135985..."
17046,"Come on, come on, turn the radio on It's Frida...","[0.35929784, 0.35824442, -0.4484742, -0.444750..."


In [9]:
import pickle
with open('./lyrics_embedding.pkl', 'wb') as f:
    pickle.dump(lyrics_data, f)